In [39]:
import sys
import re
import json
import urllib.request
import pandas as pd
import xml.etree.ElementTree as ET
sys.path.insert(0, '../scripts')
from WikiPage import WikiPage

In [21]:
dog_dat = pd.read_csv('../data/Wikipedia-dog.csv', index_col=0)
fish_dat = pd.read_csv('../data/Wikipedia-fish.csv', index_col=0)
dat = pd.concat([dog_dat, fish_dat])
dat.head()

,website,title,page_type,text_raw
970284,https://en.wikipedia.org/wiki/Category:Dog_sho...,Category:Dog shows and showing,category,{{Cat main|Conformation show|Show dog}}\n{{por...
972913,https://en.wikipedia.org/wiki/Category:Dog_health,Category:Dog health,category,This is a collection of articles about the hea...
970251,https://en.wikipedia.org/wiki/Category:Dog_org...,Category:Dog organizations,category,This is an automatically collected list of art...
729436,https://en.wikipedia.org/wiki/Category:Dog_sports,Category:Dog sports,category,This is an automatically accumulated list of a...
978163,https://en.wikipedia.org/wiki/Category:Dogs_as...,Category:Dogs as pets,category,[[Category:Dogs|Pets]]\n[[Category:Mammals as ...


In [13]:
dog_xmlfile = '../data/Wikipedia-dog.xml'
dog_tree = ET.parse(dog_xmlfile)
dog_root = dog_tree.getroot()

fish_xmlfile = '../data/Wikipedia-fish.xml'
fish_tree = ET.parse(fish_xmlfile)
fish_root = fish_tree.getroot()

In [34]:
wp = WikiPage(dog_root[30])
print(wp.text_parentid)
#dat.loc[wp.text_parentid]

685069728


In [35]:
wiki_id = str(wp.text_parentid)
request = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids=__pageids__&inprop=url&format=json"
id_request = request.replace("__pageids__", wiki_id)
s = urllib.request.urlopen(id_request).read()
article_dict = json.loads(s)
#print(str(article_dict['query']['pages'][wiki_id]['fullurl']))

In [36]:
article_dict

{'batchcomplete': '',
 'query': {'pages': {'685069728': {'missing': '', 'pageid': 685069728}}}}

In [37]:
def get_url_from_wiki_id(wiki_id):
    wiki_id = str(wiki_id)
    request = "https://en.wikipedia.org/w/api.php?action=query&prop=info&pageids=__pageids__&inprop=url&format=json"
    id_request = request.replace("__pageids__", wiki_id)
    s = urllib.request.urlopen(id_request).read()
    article_dict = json.loads(s)
    return str(article_dict['query']['pages'][wiki_id]['fullurl'])

In [76]:
text

'{{Infobox award\n| name           = PDSA Certificate for Animal Bravery or Devotion\n| current_awards = \n| image          = \n| imagesize      = 150px\n| alt            = \n| caption        = The PDSA Certificate for Animal Bravery or Devotion\n| description    = Animal gallantry and devotion to duty\n| presenter      = [[People\'s Dispensary for Sick Animals]]\n| country        =  United Kingdom\n| location       = \n| year           = 2001\n| year2          = \n| holder         = \n| website        = \n}}\n\nThe \'\'\'PDSA Certificate for Animal Bravery or Devotion\'\'\' is an animal bravery award that acknowledges the extraordinary bravery and devotion to duty of animals.<ref>{{cite web|title=PDSA Report and Accounts 2009|url=http://www.charity-commission.gov.uk/ScannedAccounts/Ends17/0000208217_ac_20091231_e_c.pdf|publisher=Charity Commission|accessdate=8 April 2011|format=PDF}}</ref>   It was created by the [[People\'s Dispensary for Sick Animals]] (PDSA) in 2001.<ref name=dotty

In [42]:
print(wp.website)

https://en.wikipedia.org/wiki/PDSA_Certificate_for_Animal_Bravery_or_Devotion


In [102]:
text = wp.text_raw

In [108]:
def find_wiki_links(text):
    searchstr = '(\[[^\[\]]*\[.*?\][^\]\[]*\])'
    link_urls = []
    link_titles = []
    link_titles_raw = re.findall(searchstr, text)
    for link_title_raw in link_titles_raw:
        link_title = re.sub('\|.+', '', link_title_raw)#remove junk at end
        link_titles.append(link_title[2:-2])
        link_title = re.sub(' ', '_',  link_title[2:-2])
        link_url = 'https://en.wikipedia.org/wiki/%s' % (link_title)
        link_urls.append(link_url)
    return list(set(zip(link_titles, link_urls)))

In [115]:
print(re.search('\|.+', 'test|remove bar').group())
print(re.sub('\|.+', '', 'test|remove bar'))

|remove bar
test


In [169]:
def query_wiki_api(title):
    base_query = 'https://en.wikipedia.org/w/api.php?action=query&titles= &prop=revisions&rvprop=content&format=json'
    query = re.sub(' ', title, base_query)
    query = re.sub(' ', '%20', query)
    result = urllib.request.urlopen(query).read()
    article_dict = json.loads(test_result)
    return article_dict

def get_text_from_wiki_api(title):
    article_dict = query_wiki_api(title)
    wiki_id = list(article_dict['query']['pages'].keys())[0]
    return article_dict['query']['pages'][wiki_id]['revisions'][0]['*']

In [171]:
test_title = find_wiki_links(text)[0][0]
print(test_title)
test_text = get_text_from_wiki_query(test_title)
print(test_text)

Labrador Retriever
{{about|the dog breed|the song by AKB48|Labrador Retriever (song)}}
{{redirect|Chocolate Lab|the record label|Chocolate Lab Records}}
{{pp-protected|small=yes}}
{{Use mdy dates|date=July 2017}}
{{EngvarB|date=July 2017}}
<!-- Please use BRITISH ENGLISH spelling in this article per article history and WP:ENGVAR -->
<!-- Begin Infobox Dogbreed. The text of the article should go AFTER this section. See: -->
<!-- http://en.wikipedia.org/wiki/Wikipedia:WikiProject_Dog_breeds#Infobox_Dogbreed_template -->
<!-- for full explanation of the syntax used in this template. -->
{{Infobox dog breed
|name     = Labrador Retriever
|image    = YellowLabradorLooking new.jpg
|image_alt = Labrador Retriever image
|image_caption = Yellow Labrador Retriever
|altname  = Labrador
|nickname = Lab
|stock = [[St. John's water dog]]
|country  = United Kingdom & Canada<ref>{{cite web|url=http://www.akc.org/dog-breeds/labrador-retriever/detail/|title=Labrador Retriever History & Training/Temperam

In [172]:
find_wiki_links(test_text)

[('Wikipedia:External links',
  'https://en.wikipedia.org/wiki/Wikipedia:External_links'),
 ('Earl of Home', 'https://en.wikipedia.org/wiki/Earl_of_Home'),
 ('Type 56 Assault Rif', 'https://en.wikipedia.org/wiki/Type_56_Assault_Rif'),
 ('Sarbi', 'https://en.wikipedia.org/wiki/Sarbi'),
 ('Sesame Street', 'https://en.wikipedia.org/wiki/Sesame_Street'),
 ('File:Labrador Retriever chocolate Hershey sit (cropped).j',
  'https://en.wikipedia.org/wiki/File:Labrador_Retriever_chocolate_Hershey_sit_(cropped).j'),
 ('recovery position', 'https://en.wikipedia.org/wiki/recovery_position'),
 ('File:LabradorWeaving.j',
  'https://en.wikipedia.org/wiki/File:LabradorWeaving.j'),
 ('chip and pin', 'https://en.wikipedia.org/wiki/chip_and_pin'),
 ('gun dog', 'https://en.wikipedia.org/wiki/gun_dog'),
 ('List of countries by populati',
  'https://en.wikipedia.org/wiki/List_of_countries_by_populati'),
 ('Avalon Peninsula', 'https://en.wikipedia.org/wiki/Avalon_Peninsula'),
 ('Peter Hawker', 'https://en.wiki

In [173]:
wiki_id = list(test_article_dict['query']['pages'].keys())[0]
link_text = test_article_dict['query']['pages'][wiki_id]['revisions'][0]['*']

In [174]:
link_text

'{{about|the dog breed|the song by AKB48|Labrador Retriever (song)}}\n{{redirect|Chocolate Lab|the record label|Chocolate Lab Records}}\n{{pp-protected|small=yes}}\n{{Use mdy dates|date=July 2017}}\n{{EngvarB|date=July 2017}}\n<!-- Please use BRITISH ENGLISH spelling in this article per article history and WP:ENGVAR -->\n<!-- Begin Infobox Dogbreed. The text of the article should go AFTER this section. See: -->\n<!-- http://en.wikipedia.org/wiki/Wikipedia:WikiProject_Dog_breeds#Infobox_Dogbreed_template -->\n<!-- for full explanation of the syntax used in this template. -->\n{{Infobox dog breed\n|name     = Labrador Retriever\n|image    = YellowLabradorLooking new.jpg\n|image_alt = Labrador Retriever image\n|image_caption = Yellow Labrador Retriever\n|altname  = Labrador\n|nickname = Lab\n|stock = [[St. John\'s water dog]]\n|country  = United Kingdom & Canada<ref>{{cite web|url=http://www.akc.org/dog-breeds/labrador-retriever/detail/|title=Labrador Retriever History & Training/Temperam

In [110]:
find_wiki_links(text)

[('Labrador Retriever', 'https://en.wikipedia.org/wiki/Labrador_Retriever'),
 ('Miniature Schnauzer', 'https://en.wikipedia.org/wiki/Miniature_Schnauzer'),
 ('Category:Domesticated animals',
  'https://en.wikipedia.org/wiki/Category:Domesticated_animals'),
 ('Stray Dog', 'https://en.wikipedia.org/wiki/Stray_Dog'),
 ('Category:Awards established in 2001',
  'https://en.wikipedia.org/wiki/Category:Awards_established_in_2001'),
 ('Lancashire', 'https://en.wikipedia.org/wiki/Lancashire'),
 ('Piper (do', 'https://en.wikipedia.org/wiki/Piper_(do'),
 ('United Kingd', 'https://en.wikipedia.org/wiki/United_Kingd'),
 ('dog', 'https://en.wikipedia.org/wiki/dog'),
 ('Yorkshire Terrier', 'https://en.wikipedia.org/wiki/Yorkshire_Terrier'),
 ('donkey', 'https://en.wikipedia.org/wiki/donkey'),
 ('Afghanistan', 'https://en.wikipedia.org/wiki/Afghanistan'),
 ("People's Dispensary for Sick Animals",
  "https://en.wikipedia.org/wiki/People's_Dispensary_for_Sick_Animals"),
 ('Improvised explosive devi',
  